<a href="https://colab.research.google.com/github/IamSVP94/OpenDataScience_course/blob/master/Homework/Homework%2010.%20Flight%20delays%20kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание № 10 (демо)<br>Прогнозирование задержек вылетов

Ваша задача – побить единственный бенчмарк в [соревновании](https://www.kaggle.com/c/flight-delays-2017) на Kaggle Inclass. Подробных инструкций не будет, будет только тезисно описано, как получен этот бенчмарк. Конечно, с помощью Xgboost. Надеюсь, на данном этапе курса вам достаточно бросить полтора взгляда на данные, чтоб понять, что это тот тип задачи, в которой затащит Xgboost. Но проверьте еще Catboost.

In [12]:
!pip install catboost

In [13]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from sklearn.metrics import roc_auc_score

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
path_to_files = '/content/drive/My Drive/Colab Notebooks/flight_delays/'
train = pd.read_csv(path_to_files+'flight_delays_train.csv')
test = pd.read_csv(path_to_files+'flight_delays_test.csv')

In [16]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [17]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


Итак, надо по времени вылета самолета, коду авиакомпании-перевозчика, месту вылета и прилета и расстоянию между аэропортами вылета и прилета предсказать задержку вылета более 15 минут. В качестве простейшего бенчмарка возьмем логистическую регрессию и два признака, которые проще всего взять: `DepTime` и `Distance`. У такой модели результат – 0.68202 на LB. 

In [18]:
X_train, y_train = train[['Distance', 'DepTime']].values, train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test[['Distance', 'DepTime']].values

X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, 
                                                                y_train, 
                                                                test_size=0.3, 
                                                                random_state=17)

In [19]:
logit = LogisticRegression(random_state=17)

logit.fit(X_train_part, y_train_part)
logit_valid_pred = logit.predict_proba(X_valid)[:, 1]

roc_auc_score(y_valid, logit_valid_pred)

0.6795697123357751

In [20]:
logit.fit(X_train, y_train)
logit_test_pred = logit.predict_proba(X_test)[:, 1]

# pd.Series(logit_test_pred, 
#           name='dep_delayed_15min').to_csv('../Data/logit_2feat.csv', 
#                                            index_label='id', 
#                                            header=True)

Как был получен бенчмарк в соревновании:
- Признаки `Distance` и  `DepTime` брались без изменений
- Создан признак "маршрут" из исходных `Origin` и `Dest`
- К признакам `Month`, `DayofMonth`, `DayOfWeek`, `UniqueCarrier` и "маршрут" применено OHE-преобразование (`LabelBinarizer`)
- Выделена отложенная выборка
- Обучалась логистическая регрессия и градиентный бустинг (xgboost), гиперпараметры бустинга настраивались на кросс-валидации, сначала те, что отвечают за сложность модели, затем число деревьев фиксировалось равным 500 и настраивался шаг градиентного спуска
- С помощью `cross_val_predict` делались прогнозы обеих моделей на кросс-валидации (именно предсказанные вероятности), настраивалась линейная смесь ответов логистической регрессии и градиентного бустинга вида $w_1 * p_{logit} + (1 - w_1) * p_{xgb}$, где $p_{logit}$ – предсказанные логистической регрессией вероятности класса 1, $p_{xgb}$ – аналогично. Вес $w_1$ подбирался вручную. 
- В качестве ответа для тестовой выборки бралась аналогичная комбинация ответов двух моделей, но уже обученных на всей обучающей выборке.

Описанный план ни к чему не обязывает – это просто то, как решение получил автор задания. Возможно, мы не захотите следовать намеченному плану, а добавите, скажем, пару хороших признаков и обучите лес из тысячи деревьев.

Удачи!

<hr>

In [21]:
from sklearn.model_selection import train_test_split

In [69]:
def df_preparation(df_train, df_test, target = 'target'):
    common = df_train.merge(df_test, how='outer', indicator=('where_is')) # объединение фреймов
    common['route'] = common['Origin'] + '-' + common['Dest']
    y = common[target].map({'Y': 1, 'N': 0})
    OHE = pd.get_dummies(common.drop([target], axis=1), prefix_sep=': ')
    
    left_indx = y[y.notna()] # индексы train
    right_indx = y[y.isna()] # индексы test

    new_df_train = OHE.loc[left_indx.index,:] # выделение df_train
    new_df_test = OHE.loc[right_indx.index,:] # выделение df_test


    return {'y_df_train': y.dropna(), 
            'df_train': new_df_train,
            'df_test': new_df_test}
 
dfs = df_preparation(train, test, target='dep_delayed_15min')

In [70]:
y_df_train = dfs['y_df_train']
df_train = dfs['df_train']
df_test = dfs['df_test']

In [71]:
df_train

,DepTime,Distance,Month: c-1,Month: c-10,Month: c-11,Month: c-12,Month: c-2,Month: c-3,Month: c-4,Month: c-5,Month: c-6,Month: c-7,Month: c-8,Month: c-9,DayofMonth: c-1,DayofMonth: c-10,DayofMonth: c-11,DayofMonth: c-12,DayofMonth: c-13,DayofMonth: c-14,DayofMonth: c-15,DayofMonth: c-16,DayofMonth: c-17,DayofMonth: c-18,DayofMonth: c-19,DayofMonth: c-2,DayofMonth: c-20,DayofMonth: c-21,DayofMonth: c-22,DayofMonth: c-23,DayofMonth: c-24,DayofMonth: c-25,DayofMonth: c-26,DayofMonth: c-27,DayofMonth: c-28,DayofMonth: c-29,DayofMonth: c-3,DayofMonth: c-30,DayofMonth: c-31,DayofMonth: c-4,...,route: TYS-MCO,route: TYS-MEM,route: TYS-MSP,route: TYS-ORD,route: TYS-TPA,route: VCT-IAH,route: VIS-FAT,route: VLD-ATL,route: VPS-ATL,route: VPS-CVG,route: VPS-DFW,route: VPS-IAH,route: VPS-MEM,route: VPS-ORD,route: WRG-KTN,route: WRG-PSG,route: WYS-SLC,route: XNA-ATL,route: XNA-CLT,route: XNA-CVG,route: XNA-DCA,route: XNA-DEN,route: XNA-DFW,route: XNA-DTW,route: XNA-EWR,route: XNA-IAH,route: XNA-LAX,route: XNA-LGA,route: XNA-MEM,route: XNA-MIA,route: XNA-MSP,route: XNA-ORD,route: XNA-SLC,route: YAK-CDV,route: YAK-JNU,route: YKM-SLC,route: YUM-IPL,route: YUM-LAS,route: YUM-LAX,route: YUM-PHX
0,1934,732,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1548,834,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1422,416,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1015,872,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1828,423,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1618,199,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
99996,804,884,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
99997,1901,1076,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
99998,1515,140,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [72]:
X_train, X_valid, y_train, y_valid = train_test_split(df_train, 
                                                      y_df_train, 
                                                      test_size=0.3, 
                                                      random_state=2,
                                                      shuffle=True)

In [64]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict

In [34]:
cat_params = {
    'depth': list(range(10),2), 
    'learning_rate': list(range())
}

In [74]:
cat_model = CatBoostClassifier(iterations=10, 
                               depth=2, 
                               learning_rate=1, 
                               loss_function='Logloss', 
                               verbose=True)

In [76]:
cat_model.fit(X_train, y_train)

0:	learn: 0.4679292	total: 93.8ms	remaining: 844ms
1:	learn: 0.4592777	total: 117ms	remaining: 468ms
2:	learn: 0.4547111	total: 140ms	remaining: 327ms
3:	learn: 0.4510000	total: 163ms	remaining: 245ms
4:	learn: 0.4491793	total: 187ms	remaining: 187ms
5:	learn: 0.4477265	total: 209ms	remaining: 140ms
6:	learn: 0.4466336	total: 232ms	remaining: 99.5ms
7:	learn: 0.4456109	total: 255ms	remaining: 63.8ms
8:	learn: 0.4445144	total: 278ms	remaining: 30.9ms
9:	learn: 0.4435457	total: 311ms	remaining: 0us


In [81]:
cat_model.get_all_params()

{'auto_class_weights': 'None',
 'bayesian_matrix_reg': 0.10000000149011612,
 'best_model_min_trees': 1,
 'boost_from_average': False,
 'boosting_type': 'Plain',
 'bootstrap_type': 'MVS',
 'border_count': 254,
 'class_names': [0, 1],
 'classes_count': 0,
 'depth': 2,
 'eval_metric': 'Logloss',
 'feature_border_type': 'GreedyLogSum',
 'grow_policy': 'SymmetricTree',
 'iterations': 10,
 'l2_leaf_reg': 3,
 'leaf_estimation_backtracking': 'AnyImprovement',
 'leaf_estimation_iterations': 10,
 'leaf_estimation_method': 'Newton',
 'learning_rate': 1,
 'loss_function': 'Logloss',
 'max_leaves': 4,
 'min_data_in_leaf': 1,
 'model_shrink_mode': 'Constant',
 'model_shrink_rate': 0,
 'model_size_reg': 0.5,
 'nan_mode': 'Min',
 'penalties_coefficient': 1,
 'random_seed': 0,
 'random_strength': 1,
 'rsm': 1,
 'sampling_frequency': 'PerTree',
 'score_function': 'Cosine',
 'sparse_features_conflict_fraction': 0,
 'subsample': 0.800000011920929,
 'task_type': 'CPU',
 'use_best_model': False}

In [77]:
y_pred = cross_val_predict(cat_model, X_valid, y_valid, cv=5)
y_pred

0:	learn: 0.4657344	total: 13.5ms	remaining: 121ms
1:	learn: 0.4619478	total: 24.8ms	remaining: 99.2ms
2:	learn: 0.4580744	total: 37.6ms	remaining: 87.6ms
3:	learn: 0.4545507	total: 48.1ms	remaining: 72.2ms
4:	learn: 0.4521091	total: 58.9ms	remaining: 58.9ms
5:	learn: 0.4492173	total: 69.9ms	remaining: 46.6ms
6:	learn: 0.4478273	total: 80.8ms	remaining: 34.6ms
7:	learn: 0.4462231	total: 91.5ms	remaining: 22.9ms
8:	learn: 0.4452253	total: 102ms	remaining: 11.4ms
9:	learn: 0.4443409	total: 115ms	remaining: 0us
0:	learn: 0.4689186	total: 14.7ms	remaining: 132ms
1:	learn: 0.4602178	total: 32.3ms	remaining: 129ms
2:	learn: 0.4569223	total: 43.2ms	remaining: 101ms
3:	learn: 0.4549445	total: 53.5ms	remaining: 80.3ms
4:	learn: 0.4536273	total: 65ms	remaining: 65ms
5:	learn: 0.4521994	total: 75.8ms	remaining: 50.6ms
6:	learn: 0.4500215	total: 86.4ms	remaining: 37ms
7:	learn: 0.4493143	total: 96.5ms	remaining: 24.1ms
8:	learn: 0.4480685	total: 109ms	remaining: 12.1ms
9:	learn: 0.4468892	total: 1

array([0., 0., 0., ..., 0., 0., 0.])

In [79]:
roc_auc_score(y_valid, y_pred)

0.5204774337933608